In [1]:
from pyspark.sql.functions import udf, col, lower, regexp_replace, concat_ws, trim
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.types import ArrayType, StringType, IntegerType

file_location = "dbfs:///FileStore/tables/all-news/*.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df_file = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location) \
  .select('id', 'title', 'content') \
  .na.drop()

# Delete punctuation
df_cleaned = df_file.select('id', (lower(regexp_replace('title', "[^a-zA-Z\\s]", " ")).alias('title')), \
                                  (lower(regexp_replace('content', "[^a-zA-Z\\s]", " ")).alias('content')))

df_cleaned = df_cleaned.select('id', (regexp_replace('title', "[!-~]?\\b[\\w]\\b[!-~]?", " ")).alias('title'), \
                                     (regexp_replace('content', "[!-~]?\\b[\\w]\\b[!-~]?", " ")).alias('content'))

df_cleaned = df_cleaned.select('id', (regexp_replace(trim(col('title')), " +", " ")).alias('title'), \
                                     (regexp_replace(trim(col('content')), " +", " ")).alias('content'))

# Tokenize title
title_tokenizer = Tokenizer(inputCol='title', outputCol='tokenized_title')
df_tokenized_title = title_tokenizer.transform(df_cleaned).select('id', 'tokenized_title', 'content')

# Remove stopwords from title
stopwords_title_remover = StopWordsRemover(inputCol='tokenized_title', outputCol='cleaned_title')
df_title_removed_stopwords = stopwords_title_remover.transform(df_tokenized_title).select('id', 'cleaned_title', 'content')

# Clean words whose lenght is less than 1
filter_length_udf = udf(lambda row: [x for x in row if len(x) > 1], ArrayType(StringType()))
df_final_title = df_title_removed_stopwords.withColumn('cleaned_title', filter_length_udf(col('cleaned_title')))

# Tokenize content
content_tokenizer = Tokenizer(inputCol='content', outputCol='tokenized_content')
df_tokenized_content = content_tokenizer.transform(df_final_title).select('id', 'cleaned_title', 'tokenized_content')

# Remove stopwords from content
stopwords_remover = StopWordsRemover(inputCol='tokenized_content', outputCol='cleaned_content')
df_removed_stopwords = stopwords_remover.transform(df_tokenized_content).select('id', 'cleaned_title', 'cleaned_content')

# Filter length in content
df_final = df_removed_stopwords.withColumn('cleaned_content', filter_length_udf(col('cleaned_content')))

# Make title and content strings and id an integer
df_final = df_final.withColumn('cleaned_title', concat_ws(" ", 'cleaned_title')) \
           .withColumn('cleaned_content', concat_ws(" ", 'cleaned_content')) \
           .withColumn('id', df_final['id'].cast(IntegerType())) \
           .select('id', col('cleaned_title').alias('title'), col('cleaned_content').alias('content')) 
           
display(df_final)

id title content 17283 house republicans fret winning health care suit new york times washington congressional republicans new fear comes health care lawsuit obama administration might win incoming trump administration choose longer defend executive branch suit challenges administration authority spend billions dollars health insurance subsidies americans handing house republicans big victory issues sudden loss disputed subsidies conceivably cause health care program implode leaving millions people without access health insurance republicans prepared replacement lead chaos insurance market spur political backlash republicans gain full control government stave outcome republicans find awkward position appropriating huge sums temporarily prop obama health care law angering conservative voters demanding end law years another twist donald trump administration worried preserving executive branch prerogatives choose fight republican allies house central questions dispute eager avoid ugly political pileup republicans capitol hill trump transition team gaming handle lawsuit election put limbo least late february united states court appeals district columbia circuit yet ready divulge strategy given pending litigation involves obama administration congress inappropriate comment said phillip blando spokesman trump transition effort upon taking office trump administration evaluate case related aspects affordable care act potentially decision judge rosemary collyer ruled house republicans standing sue executive branch spending dispute obama administration distributing health insurance subsidies violation constitution without approval congress justice department confident judge collyer decision reversed quickly appealed subsidies remained place appeal successfully seeking temporary halt proceedings mr trump won house republicans last month told court transition team currently discussing potential options resolution matter take effect inauguration jan suspension case house lawyers said provide future administration time consider whether continue prosecuting otherwise resolve appeal republican leadership officials house acknowledge possibility cascading effects payments totaled estimated billion suddenly stopped insurers receive subsidies exchange paying costs deductibles eligible consumers race drop coverage since losing money loss subsidies destabilize entire program cause lack confidence leads insurers seek quick exit well anticipating trump administration might inclined mount vigorous fight house republicans given dim view health care law team lawyers month sought intervene case behalf two participants health care program request lawyers predicted deal house republicans new administration dismiss settle case produce devastating consequences individuals receive reductions well nation health insurance health care systems generally matter happens house republicans say want prevail two overarching concepts congressional power purse right congress sue executive branch violates constitution regarding spending power house republicans contend congress never appropriated money subsidies required constitution suit initially championed john boehner house speaker time later house committee reports republicans asserted administration desperate funding required treasury department provide despite widespread internal skepticism spending proper white house said spending permanent part law passed annual appropriation required even though administration initially sought one important house republicans judge collyer found congress standing sue white house issue ruling many legal experts said flawed want precedent set restore congressional leverage executive branch spending power standing trump administration may come pressure advocates presidential authority fight house matter shared views health care since precedents broad repercussions complicated set dynamics illustrating quick legal victory house trump era might come costs republicans never anticipated 

In [2]:
articles_rdd = df_final.rdd.map(lambda x: (x['id'], x['title'], x['content']))
print(articles_rdd.take(5))

[(17283, 'house republicans fret winning health care suit new york times', 'washington congressional republicans new fear comes health care lawsuit obama administration might win incoming trump administration choose longer defend executive branch suit challenges administration authority spend billions dollars health insurance subsidies americans handing house republicans big victory issues sudden loss disputed subsidies conceivably cause health care program implode leaving millions people without access health insurance republicans prepared replacement lead chaos insurance market spur political backlash republicans gain full control government stave outcome republicans find awkward position appropriating huge sums temporarily prop obama health care law angering conservative voters demanding end law years another twist donald trump administration worried preserving executive branch prerogatives choose fight republican allies house central questions dispute eager avoid ugly political pileup republicans capitol hill trump transition team gaming handle lawsuit election put limbo least late february united states court appeals district columbia circuit yet ready divulge strategy given pending litigation involves obama administration congress inappropriate comment said phillip blando spokesman trump transition effort upon taking office trump administration evaluate case related aspects affordable care act potentially decision judge rosemary collyer ruled house republicans standing sue executive branch spending dispute obama administration distributing health insurance subsidies violation constitution without approval congress justice department confident judge collyer decision reversed quickly appealed subsidies remained place appeal successfully seeking temporary halt proceedings mr trump won house republicans last month told court transition team currently discussing potential options resolution matter take effect inauguration jan suspension case house lawyers said provide future administration time consider whether continue prosecuting otherwise resolve appeal republican leadership officials house acknowledge possibility cascading effects payments totaled estimated billion suddenly stopped insurers receive subsidies exchange paying costs deductibles eligible consumers race drop coverage since losing money loss subsidies destabilize entire program cause lack confidence leads insurers seek quick exit well anticipating trump administration might inclined mount vigorous fight house republicans given dim view health care law team lawyers month sought intervene case behalf two participants health care program request lawyers predicted deal house republicans new administration dismiss settle case produce devastating consequences individuals receive reductions well nation health insurance health care systems generally matter happens house republicans say want prevail two overarching concepts congressional power purse right congress sue executive branch violates constitution regarding spending power house republicans contend congress never appropriated money subsidies required constitution suit initially championed john boehner house speaker time later house committee reports republicans asserted administration desperate funding required treasury department provide despite widespread internal skepticism spending proper white house said spending permanent part law passed annual appropriation required even though administration initially sought one important house republicans judge collyer found congress standing sue white house issue ruling many legal experts said flawed want precedent set restore congressional leverage executive branch spending power standing trump administration may come pressure advocates presidential authority fight house matter shared views health care since precedents broad repercussions complicated set dynamics illustrating quick legal victory house trump era might come costs republicans never anticipated took obama

In [3]:
# Inverted index

import itertools
import operator

def accumulate(l):
  it = itertools.groupby(l, operator.itemgetter(0))
  for key, subiter in it:
     yield key, sum(item[1] for item in subiter)


inverted_index_rdd = articles_rdd.flatMap(lambda line: [(word , (line[0], 1)) for word in (line[1] + " " + line[2]).split(" ")]) \
                                 .groupByKey() \
                                 .map(lambda word: (word[0], list(word[1]))) \
                                 .map(lambda lista: (lista[0], sorted(list(accumulate(lista[1])), key = lambda x: -x[1]))) \
                                 .cache()


inverted_index_rdd.take(5)

Out[ 3 ]: 
[('sensitisation', [(145965, 1)]),
 ('eftersom', [(46868, 1)]),
 ('ehning', [(54324, 3)]),
 ('avis',
 [(19086, 6),
 (175175, 3),
 (None, 2),
 (21022, 1),
 (21537, 1),
 (22811, 1),
 (23932, 1),
 (43766, 1),
 (46197, 1),
 (52857, 1),
 (75082, 1),
 (80066, 1),
 (189553, 1),
 (197133, 1),
 (134572, 1),
 (135331, 1),
 (140388, 1),
 (142943, 1),
 (147717, 1)]),
 ('usine', [(33345, 1)])]

In [4]:
# Online

new_df = df_file.withColumn('id', df_file['id'].cast(IntegerType())) \
                .select('id', 'title', 'content') 

file_rdd = new_df.rdd.map(lambda x: (x['id'], x['title']))
file_map = file_rdd.collectAsMap()

In [5]:
dbutils.widgets.text("word", "Please enter word to search")
dbutils.widgets.text("search", "Please enter id to search")

In [6]:
toSearch = str(dbutils.widgets.get("word"))
final_result = inverted_index_rdd.filter(lambda x, toSearch=toSearch: x[0] == toSearch) \
                                 .flatMap(lambda result: result[1])

final_result_list = final_result.collect()
print(final_result_list)

[(157218, 37), (21637, 35), (22019, 18), (80078, 18), (172524, 17), (None, 15), (152630, 14), (153519, 14), (81994, 12), (67320, 11), (20670, 10), (21956, 10), (52043, 10), (152370, 10), (152725, 10), (156506, 10), (190400, 10), (21938, 9), (25214, 9), (56559, 9), (167381, 9), (186035, 9), (202471, 9), (71258, 9), (21429, 8), (57134, 8), (164430, 8), (167357, 8), (171843, 8), (172671, 8), (191001, 8), (198944, 8), (None, 8), (146054, 8), (70542, 8), (57626, 7), (59310, 7), (78796, 7), (80442, 7), (156494, 7), (168775, 7), (173338, 7), (190398, 7), (190411, 7), (193556, 7), (203825, 7), (20237, 6), (25996, 6), (32497, 6), (52978, 6), (80649, 6), (80812, 6), (153549, 6), (153694, 6), (170202, 6), (170348, 6), (172988, 6), (191066, 6), (64052, 6), (71053, 6), (72473, 6), (19805, 5), (23303, 5), (28682, 5), (42312, 5), (45251, 5), (51834, 5), (52974, 5), (58884, 5), (59693, 5), (80713, 5), (81300, 5), (167206, 5), (170221, 5), (174602, 5), (186097, 5), (190277, 5), (191000, 5), (192082, 5), (192872, 5), (197685, 5), (198660, 5), (202520, 5), (146187, 5), (147477, 5), (71851, 5), (73163, 5), (18295, 4), (18559, 4), (20117, 4), (22325, 4), (22589, 4), (25974, 4), (28274, 4), (32957, 4), (35666, 4), (36914, 4), (38158, 4), (38201, 4), (39772, 4), (41606, 4), (41660, 4), (43784, 4), (45410, 4), (51474, 4), (52943, 4), (58069, 4), (59383, 4), (74685, 4), (80907, 4), (152863, 4), (152905, 4), (156136, 4), (172815, 4), (173090, 4), (174359, 4), (174408, 4), (174543, 4), (177288, 4), (185711, 4), (190909, 4), (190971, 4), (191014, 4), (191753, 4), (192135, 4), (193184, 4), (193809, 4), (146776, 4), (65437, 4), (22581, 3), (24478, 3), (24522, 3), (24533, 3), (25461, 3), (27903, 3), (29708, 3), (33004, 3), (39955, 3), (39996, 3), (41143, 3), (48866, 3), (55106, 3), (58238, 3), (58294, 3), (59380, 3), (59670, 3), (78780, 3), (78894, 3), (82116, 3), (153612, 3), (156373, 3), (157552, 3), (157851, 3), (163449, 3), (167092, 3), (167187, 3), (167263, 3), (168515, 3), (169825, 3), (170546, 3), (171526, 3), (172122, 3), (172737, 3), (174374, 3), (175176, 3), (180232, 3), (185351, 3), (185849, 3), (193964, 3), (194747, 3), (199005, 3), (201969, 3), (71186, 3), (19005, 2), (22030, 2), (23075, 2), (24520, 2), (24574, 2), (24949, 2), (24964, 2), (25312, 2), (25497, 2), (26009, 2), (26486, 2), (28681, 2), (28829, 2), (32112, 2), (32504, 2), (35813, 2), (38177, 2), (43641, 2), (44417, 2), (46435, 2), (47945, 2), (49012, 2), (52168, 2), (54096, 2), (54307, 2), (57396, 2), (58240, 2), (59339, 2), (59352, 2), (59381, 2), (59471, 2), (59488, 2), (73942, 2), (78611, 2), (80067, 2), (80648, 2), (80696, 2), (80786, 2), (81002, 2), (82558, 2), (155831, 2), (155980, 2), (156214, 2), (156220, 2), (156393, 2), (156522, 2), (161124, 2), (166746, 2), (167156, 2), (167626, 2), (169973, 2), (170166, 2), (171490, 2), (172051, 2), (172569, 2), (173382, 2), (174230, 2), (174254, 2), (175325, 2), (176479, 2), (180181, 2), (180543, 2), (182433, 2), (187963, 2), (189782, 2), (190713, 2), (193842, 2), (193998, 2), (194710, 2), (198008, 2), (198834, 2), (135149, 2), (139220, 2), (144164, 2), (145765, 2), (147445, 2), (148358, 2), (149787, 2), (150191, 2), (151722, 2), (71352, 2), (17366, 1), (17416, 1), (17430, 1), (17663, 1), (17719, 1), (18613, 1), (18655, 1), (18749, 1), (19116, 1), (19358, 1), (19410, 1), (19792, 1), (19815, 1), (19831, 1), (20146, 1), (20451, 1), (20523, 1), (20652, 1), (21257, 1), (21313, 1), (21408, 1), (21829, 1), (22043, 1), (22053, 1), (22202, 1), (22452, 1), (22477, 1), (22512, 1), (22563, 1), (22567, 1), (22728, 1), (22982, 1), (23117, 1), (23225, 1), (23311, 1), (23333, 1), (23675, 1), (24544, 1), (24553, 1), (24605, 1), (24954, 1), (25154, 1), (25363, 1), (25385, 1), (25502, 1), (25547, 1), (25569, 1), (26296, 1), (28100, 1), (28476, 1), (28974, 1), (29294, 1), (29967, 1), (30013, 1), (30756, 1), (31432, 1), (31809, 1), (32022, 1), (32044, 1), (32679, 1), (33742, 1), (34513, 1), (34820, 1), (34836, 1), (34888, 1), (34905, 1), (35030, 1), (35286,

In [7]:
def printing_result(): 
  cont = 0
  maximum = 5
  for i in final_result_list:
    if cont == maximum: break
    if i[0] != None:
      cont += 1
      yield i[1], list(((k, v) for k, v in file_map.items() if k == i[0]))

print(list(printing_result()))

[(37, [(157218, 'Peru 0-0 Colombia (Colombia win 4-2 on penalties): Copa América – as it happened')]), (35, [(21637, 'The Secret History of Colombia’s Paramilitaries and the U.S. War on Drugs - The New York Times')]), (18, [(22019, 'Colombian Opposition to Peace Deal Feeds Off Gay Rights Backlash - The New York Times')]), (18, [(80078, 'The End of Colombian Exceptionalism')]), (17, [(172524, 'Can Colombia Finally Fix Its Split Personality? ')])]

In [8]:
from collections import defaultdict

def accumulate2(l):
  d = defaultdict(list)
  for k, *v in l:
    d[k].append(sum(v))
  for k in d.keys():
    yield k, len(d[k])

news_rdd = articles_rdd.flatMap(lambda line: [(line[0] , (word, 1)) for word in (line[1] + " " + line[2]).split(" ")]) \
                       .groupByKey() \
                       .map(lambda word: (word[0], list(word[1]))) \
                       .filter(lambda x: x[0] != None) \
                       .map(lambda lista: (lista[0], sorted(list(accumulate2(lista[1])), key = lambda x: -x[1]))) \
                       .cache()

news_rdd.count()

Out[ 8 ]: 102038

In [9]:
from functools import reduce

id_search = int(dbutils.widgets.get("search"))
if not id_search in file_map or id_search == None: print("Not found")
else:
  new_title = file_map[id_search]
  in_new_rdd = news_rdd.filter(lambda x, id_search=id_search: x[0] == id_search)  
  in_new_list = in_new_rdd.collect()
  
  def news_similarity2(rdd_other_news):
    list1 = in_new_list[0][1]
    list2 = rdd_other_news[1]
    list3 = []
    for value in list1:
      for v in list2:
        if value[0] == v[0]: list3.append(value[0]) 
    union = list1 + list2
    distance_list = list(filter((lambda x, list3=list3: x[0] in list3), union))
    if len(distance_list) != 0:
      last_result = reduce(lambda a, b: (a[0], a[1] + b[1]) if a[0] != "" and b[0] != "" else 0, distance_list)[1]
      result = [rdd_other_news[0], last_result, len(list3)]
    else:
      result = [rdd_other_news[0], 0, 0]
    return result

  other_news = news_rdd.filter(lambda x, id_search=id_search: x[0] != id_search) 
  sim_news = other_news.map(news_similarity2) \
                          .sortBy(lambda x: -x[1])
                          
  sim_news_df = sim_news.toDF(["id","similarity","words"])
  display(sim_news_df)

id,similarity,words
null,58485,231
36274,1035,110
51752,995,82
18600,995,82
25704,991,109
24476,982,84
75718,954,112
24673,937,120
75386,921,113
75323,915,110


In [10]:
aux = sim_news.take(5)
news_result_final = []
for i in aux:
  news_result_final.append(i[0])
print(id_search, new_title, news_result_final)

167499 Reporting On The Zika Virus Means Getting Up Close And Personal [None, 36274, 51752, 18600, 25704]